# Load Data

Load raw single cell RNA-seq and necessary annotations for SCENIC pipeline application

-----

First, some technicalities 

In [1]:
import pandas as pd
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri

In [2]:
# Enable pandas conversion for R data frames
pandas2ri.activate()

#### Source my R functions that handle Seurat objects

In [3]:
try:
    # Source the R script
    robjects.r.source("../scripts/data_loader.R")
    print("R script sourced successfully!")
except Exception as e:
    print("Error sourcing R script:", e)


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

R[write to console]: Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat

R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


R[write to console]: Attaching SeuratObject

R[write to console]: Attaching sp



R script sourced successfully!


In [4]:
load_seurat = robjects.globalenv["load_seurat"]
extract_raw_counts = robjects.globalenv["extract_raw_counts"]
load_cell_types = robjects.globalenv["load_cell_types"]

#### Load Seurat file (not public data) with R function

In [5]:
%%capture

seurat_obj = load_seurat("../data/raw/SeuratProject.h5Seurat")

Load raw RNA-seq counts table (and transform to pandas)

In [6]:
raw_counts_r = extract_raw_counts(seurat_obj)
# Convert the R data frame to a pandas DataFrame
raw_counts = pandas2ri.rpy2py_dataframe(raw_counts_r)
# print(raw_counts.head())

Load cell type annotations for all cells in raw counts

In [7]:
cell_types_r = load_cell_types(seurat_obj)
cell_types = pandas2ri.rpy2py_dataframe(cell_types_r)
# print(cell_types.head())

### Save transformed data as CSV for later use 

In [16]:
raw_counts.to_csv("../data/raw/expression_mat.csv")
cell_types.to_csv("../data/annotations/cell_types.csv")

-------

### Additional information necessary for SCENIC

Species specific Transcription Factors names. I used: https://github.com/aertslab/pySCENIC/blob/master/resources/allTFs_dmel.txt (29.3.2023)

Save Drosophila TFs names as annotations data

In [ ]:
!wget -O ../data/external/allTFs_dmel.txt https://github.com/aertslab/pySCENIC/blob/d2309fed83759a0e66e0a995ef3d2f40bf4d6c1a/resources/allTFs_dmel.txt 

--2024-12-23 11:38:49--  https://github.com/aertslab/pySCENIC/blob/d2309fed83759a0e66e0a995ef3d2f40bf4d6c1a/resources/allTFs_dmel.txt
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘../data/annotations/allTFs_dmel.txt’

../data/annotations     [ <=>                ] 499.13K  --.-KB/s    in 0.1s    

2024-12-23 11:38:49 (3.84 MB/s) - ‘../data/annotations/allTFs_dmel.txt’ saved [511106]



Ranking databases for motif enrichment: https://resources.aertslab.org/cistarget/ (https://resources.aertslab.org/cistarget/databases/drosophila_melanogaster/dm6/flybase_r6.02/mc8nr/gene_based/)



In [ ]:
!wget -O ../data/external/genes_vs_motifs.rankings.feather https://resources.aertslab.org/cistarget/databases/drosophila_melanogaster/dm6/flybase_r6.02/mc8nr/gene_based/dm6-5kb-upstream-full-tx-11species.mc8nr.genes_vs_motifs.rankings.feather

--2024-12-23 11:47:30--  https://resources.aertslab.org/cistarget/databases/drosophila_melanogaster/dm6/flybase_r6.02/mc8nr/gene_based/dm6-5kb-upstream-full-tx-11species.mc8nr.genes_vs_motifs.rankings.feather
Resolving resources.aertslab.org (resources.aertslab.org)... 134.58.50.9
Connecting to resources.aertslab.org (resources.aertslab.org)|134.58.50.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 782107954 (746M)
Saving to: ‘../data/forScenic/genes_vs_motifs.rankings.feather’

../data/forScenic/g 100%[===================>] 745.88M  12.3MB/s    in 60s     

2024-12-23 11:48:31 (12.4 MB/s) - ‘../data/forScenic/genes_vs_motifs.rankings.feather’ saved [782107954/782107954]



Motif annotations file: https://resources.aertslab.org/cistarget/motif2tf/ (v8 matches mc8nr db)

In [19]:
!wget -O ../data/external/flybase_motifs_annotations.tbl https://resources.aertslab.org/cistarget/motif2tf/motifs-v8-nr.flybase-m0.001-o0.0.tbl

--2024-12-23 12:04:32--  https://resources.aertslab.org/cistarget/motif2tf/motifs-v8-nr.flybase-m0.001-o0.0.tbl
Resolving resources.aertslab.org (resources.aertslab.org)... 134.58.50.9
Connecting to resources.aertslab.org (resources.aertslab.org)|134.58.50.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40177800 (38M)
Saving to: ‘../data/external/flybase_motifs_annotations.tbl’

../data/external/fl 100%[===================>]  38.32M  11.9MB/s    in 3.2s    

2024-12-23 12:04:35 (11.9 MB/s) - ‘../data/external/flybase_motifs_annotations.tbl’ saved [40177800/40177800]

